In [68]:
import pandas as pd
import numpy as np

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, root_mean_squared_error 
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import roc_curve, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report 
from sklearn.preprocessing import StandardScaler
from lazypredict.Supervised import LazyRegressor

<h1 style="color: #FF8C00;">01 | Data Extraction </h1>

In [69]:
data = pd.read_csv("./SP500_1D.csv")
df = data.copy()


In [70]:
df.head(3)

,time,open,high,low,close,VWAP,Upper,Basis,Lower,EMA,...,RSI,RSI-based MA,Regular Bullish,Regular Bullish Label,Regular Bearish,Regular Bearish Label,Histogram,MACD,Signal,MOM
0,2012-03-30,1398.00,1406.25,1395.75,1403.25,1343.34,1401.04,1370.81,1340.58,1389.44,...,61.72,66.11,NaN,NaN,NaN,NaN,-1.09,13.64,14.73,4.75
1,2012-04-02,1407.25,1417.75,1399.00,1412.50,1344.42,1403.91,1372.95,1341.99,1391.63,...,65.31,65.87,NaN,NaN,NaN,NaN,-0.69,13.87,14.55,8.50
2,2012-04-03,1412.00,1414.25,1399.50,1408.75,1345.40,1405.60,1374.79,1343.97,1393.26,...,62.74,65.54,NaN,NaN,NaN,NaN,-0.77,13.59,14.36,8.75


In [71]:
df.shape

(3272, 27)

In [72]:
df.columns = [column.lower().replace(' ', '_') for column in data.columns]

In [73]:
duplicate_rows = df.duplicated().sum()
duplicate_rows


0

In [74]:
nan_values = df.isna().sum()
nan_values


time                        0
open                        0
high                        0
low                         0
close                       0
vwap                        0
upper                       0
basis                       0
lower                       0
ema                         0
ema.1                       0
ema.2                       0
ema.3                       0
volume                      0
volume_ma                   0
up_trend                 1218
down_trend               2054
rsi                         0
rsi-based_ma                0
regular_bullish          3272
regular_bullish_label    3272
regular_bearish          3272
regular_bearish_label    3272
histogram                   0
macd                        0
signal                      0
mom                         0
dtype: int64

In [75]:
empty_spaces = df.eq(' ').sum()
empty_spaces

time                     0
open                     0
high                     0
low                      0
close                    0
vwap                     0
upper                    0
basis                    0
lower                    0
ema                      0
ema.1                    0
ema.2                    0
ema.3                    0
volume                   0
volume_ma                0
up_trend                 0
down_trend               0
rsi                      0
rsi-based_ma             0
regular_bullish          0
regular_bullish_label    0
regular_bearish          0
regular_bearish_label    0
histogram                0
macd                     0
signal                   0
mom                      0
dtype: int64

In [76]:
df.dtypes

time                      object
open                     float64
high                     float64
low                      float64
close                    float64
vwap                     float64
upper                    float64
basis                    float64
lower                    float64
ema                      float64
ema.1                    float64
ema.2                    float64
ema.3                    float64
volume                     int64
volume_ma                float64
up_trend                 float64
down_trend               float64
rsi                      float64
rsi-based_ma             float64
regular_bullish          float64
regular_bullish_label    float64
regular_bearish          float64
regular_bearish_label    float64
histogram                float64
macd                     float64
signal                   float64
mom                      float64
dtype: object

<h1 style="color: #FF8C00;">02 | Data Cleaning </h1>

<h2 style="color: #FF6347;">Dropping unnecessary columns</h2>

In [77]:
columns = df.columns.tolist()
columns

['time',
 'open',
 'high',
 'low',
 'close',
 'vwap',
 'upper',
 'basis',
 'lower',
 'ema',
 'ema.1',
 'ema.2',
 'ema.3',
 'volume',
 'volume_ma',
 'up_trend',
 'down_trend',
 'rsi',
 'rsi-based_ma',
 'regular_bullish',
 'regular_bullish_label',
 'regular_bearish',
 'regular_bearish_label',
 'histogram',
 'macd',
 'signal',
 'mom']

In [78]:
df = df.rename(columns={'ema': 'ema20', 'ema.1': 'ema200', 'ema.2': 'ema50', 'ema.3': 'ema9'})

In [79]:
columns_to_delete = ['regular_bullish',
 'regular_bullish_label',
 'regular_bearish',
 'regular_bearish_label',
 'macd',
 'signal']

df = df.drop(columns=columns_to_delete)
df.head(2)

,time,open,high,low,close,vwap,upper,basis,lower,ema20,...,ema50,ema9,volume,volume_ma,up_trend,down_trend,rsi,rsi-based_ma,histogram,mom
0,2012-03-30,1398.00,1406.25,1395.75,1403.25,1343.34,1401.04,1370.81,1340.58,1389.44,...,1358.65,1399.42,1619524,1682877.10,1368.32,NaN,61.72,66.11,-1.09,4.75
1,2012-04-02,1407.25,1417.75,1399.00,1412.50,1344.42,1403.91,1372.95,1341.99,1391.63,...,1360.76,1402.03,1718652,1692012.90,1368.32,NaN,65.31,65.87,-0.69,8.50


In [80]:
df.sample(10)

,time,open,high,low,close,vwap,upper,basis,lower,ema20,...,ema50,ema9,volume,volume_ma,up_trend,down_trend,rsi,rsi-based_ma,histogram,mom
1510,2018-03-27,2659.50,2679.75,2596.00,2615.75,2722.07,2816.96,2712.23,2607.50,2700.86,...,2711.27,2673.01,2448450,1881209.45,NaN,2761.39,37.26,48.01,-14.73,-157.00
2889,2023-09-14,4521.25,4562.00,4519.00,4555.00,4215.55,4570.16,4479.16,4388.16,4489.57,...,4465.79,4503.95,1640992,1562933.80,4404.01,NaN,61.06,52.80,9.21,30.75
1919,2019-11-07,3077.25,3097.00,3070.25,3086.00,2863.93,3057.63,3004.08,2950.53,3029.83,...,2993.15,3058.57,1353330,1181649.25,3003.30,NaN,68.44,62.18,6.21,81.75
2035,2020-04-27,2829.00,2881.25,2812.75,2869.00,2928.68,2979.41,2767.94,2556.46,2747.64,...,2802.94,2797.67,1224445,1839590.55,2529.79,NaN,57.20,54.20,24.21,109.75
2902,2023-10-03,4326.00,4335.75,4251.25,4264.75,4227.25,4538.13,4422.87,4307.62,4391.16,...,4426.44,4340.53,2256808,1756018.10,NaN,4466.35,29.34,40.49,-16.72,-225.25
376,2013-09-25,1692.25,1695.00,1684.50,1685.75,1592.86,1707.21,1675.00,1642.79,1683.91,...,1669.88,1693.06,1639285,1682098.70,1669.46,NaN,52.91,60.83,1.07,-3.00
1259,2017-03-28,2340.50,2360.50,2333.50,2351.50,2321.21,2380.97,2341.93,2302.88,2354.49,...,2330.49,2351.55,1623900,1623990.70,2336.02,NaN,50.29,55.41,-6.01,-11.50
1652,2018-10-17,2816.25,2824.25,2783.25,2816.25,2754.26,2949.14,2861.87,2774.60,2850.69,...,2858.69,2818.49,1948526,1826659.40,NaN,2876.03,41.80,43.71,-12.96,-115.25
110,2012-09-05,1406.25,1408.25,1397.25,1403.50,1350.17,1414.89,1388.54,1362.20,1400.53,...,1382.15,1404.95,1429039,1359377.80,1374.19,NaN,55.54,61.39,-2.75,-9.00
1140,2016-10-05,2144.25,2158.00,2141.00,2153.25,2044.08,2198.95,2153.20,2107.46,2152.76,...,2150.34,2151.72,1369566,1909474.35,NaN,2198.80,49.87,48.51,1.01,-3.00


<h1 style="color: #FF8C00;">03 | Feature and Target Engineering Part1 </h1>

In [81]:
# up trend = 1, down trend = 0
df['up_trend'] = np.where(df['up_trend'].notna(), 1, 0)
df = df.rename(columns={'up_trend': 'trend_direction'})

In [82]:
df = df.drop(columns='down_trend')

In [83]:
df['1_day_trend'] = np.where(df['close'] > df['close'].shift(1), 1, 0)

In [84]:
target_df = df[['time','close']]
target_df = target_df.rename(columns={'close': 'close_tomorrow'})
target_df

,time,close_tomorrow
0,2012-03-30,1403.25
1,2012-04-02,1412.50
2,2012-04-03,1408.75
3,2012-04-04,1393.25
4,2012-04-05,1390.25
...,...,...
3267,2025-03-18,5669.25
3268,2025-03-19,5729.75
3269,2025-03-20,5712.75
3270,2025-03-21,5718.25


In [85]:
target_df = target_df.drop(index=0)

target_df.reset_index(drop=True, inplace=True)

In [86]:
df = df.drop(index=3271)

df.reset_index(drop=True, inplace=True)

I bring both data sets to match the rows. Data from today should predict the data from tomorrow.

In [87]:
df.tail(2)

,time,open,high,low,close,vwap,upper,basis,lower,ema20,...,ema50,ema9,volume,volume_ma,trend_direction,rsi,rsi-based_ma,histogram,mom,1_day_trend
3269,2025-03-20,5731.75,5765.25,5682.50,5712.75,5915.70,6075.44,5855.49,5635.55,5770.48,...,5882.92,5694.14,1671460,2264132.60,0,42.89,35.12,3.42,-33.50,0
3270,2025-03-21,5715.25,5723.75,5651.25,5718.25,5912.37,6060.90,5848.46,5636.01,5765.51,...,5876.46,5698.96,1486363,2241422.60,0,43.34,35.63,8.65,-57.75,1


In [88]:
target_df.tail(2)

,time,close_tomorrow
3269,2025-03-21,5718.25
3270,2025-03-24,5801.00


In [89]:
df = df.drop(columns='time')

In [90]:
target_df = target_df.drop(columns='time')

In [91]:
df_all = pd.concat([df, target_df], axis=1)
df_all.head(3)

,open,high,low,close,vwap,upper,basis,lower,ema20,ema200,...,ema9,volume,volume_ma,trend_direction,rsi,rsi-based_ma,histogram,mom,1_day_trend,close_tomorrow
0,1398.00,1406.25,1395.75,1403.25,1343.34,1401.04,1370.81,1340.58,1389.44,1287.55,...,1399.42,1619524,1682877.10,1,61.72,66.11,-1.09,4.75,0,1412.50
1,1407.25,1417.75,1399.00,1412.50,1344.42,1403.91,1372.95,1341.99,1391.63,1288.80,...,1402.03,1718652,1692012.90,1,65.31,65.87,-0.69,8.50,1,1408.75
2,1412.00,1414.25,1399.50,1408.75,1345.40,1405.60,1374.79,1343.97,1393.26,1289.99,...,1403.38,1673086,1653121.70,1,62.74,65.54,-0.77,8.75,0,1393.25


In [92]:
df_all.corr()['close_tomorrow'].sort_values(ascending=True)

volume_ma         -0.08
volume            -0.04
histogram         -0.01
1_day_trend        0.01
trend_direction    0.04
rsi                0.05
rsi-based_ma       0.06
mom                0.09
vwap               0.99
ema200             0.99
upper              1.00
ema50              1.00
basis              1.00
lower              1.00
ema20              1.00
ema9               1.00
open               1.00
high               1.00
low                1.00
close              1.00
close_tomorrow     1.00
Name: close_tomorrow, dtype: float64

<h1 style="color: #FF8C00;">04 | Modeling</h1>

In [94]:
X = df_all.drop("close_tomorrow", axis=1)
y = df_all["close_tomorrow"]

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [96]:
print(f'100% of our data: {len(df)}.')
print(f'70% for training data: {len(X_train)}.')
print(f'30% for test data: {len(X_test)}.')

100% of our data: 3271.
70% for training data: 2616.
30% for test data: 655.


In [99]:
regressor = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)

models, predictions = regressor.fit(X_train, X_test, y_train, y_test)

print(models.head(20)) 

100%|██████████| 42/42 [00:08<00:00,  5.22it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000231 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4596
[LightGBM] [Info] Number of data points in the train set: 2616, number of used features: 20
[LightGBM] [Info] Start training from score 3068.055428
                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
OrthogonalMatchingPursuit                    1.00       1.00 32.70        0.00
OrthogonalMatchingPursuitCV                  1.00       1.00 32.71        0.01
LassoLarsCV                                  1.00       1.00 32.80        0.02
LassoLars                                    1.00       1.00 32.80        0.00
LinearRegression                             1.00       1.00 32.94        0.01
TransformedTargetRegressor                   1.00       1.00 32.94        0.01
RANSACRegressor         

In [ ]:
df_all['difference'] = df_all['close_tomorrow'] - df_all['close']

mean_absolute_difference = df_all['difference'].abs().mean()
mean_absolute_difference 

21.710562519107306